In [159]:
import pandas as pd
import glob
import os

In [160]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import BayesianRidge , LinearRegression

In [68]:
# change into the directory where the Testing and Training data are located
wd = os.path.abspath("TrainTestingData")

# read all files into an dataframe and save dataframes to the df_list
training_set = wd + "/training_set_with_features.csv"
testing_set = wd + "/testing_set_with_features.csv"
df_train = pd.read_csv(training_set)
df_test = pd.read_csv(testing_set)

try:
    del df_test["DATE"]
    del df_train["DATE"]
except:
    print("Already deleted the 'DATE'column in both df_test and df_train dataframes")

### Below is the subsequence function, which uses an algorithm to find ALL POSSIBLE subsequence combinations and save them into the have_list set variable so we can later loop through all of them and apply them to different models, to calculate and find the combination of features that yields to the best performance. 

In [70]:

# have_list = set()

# def subsequence(array):
#     array = list(array)
#     for i in range(len(array)):
#         j = len(array)
#         while j > i:
#             sub_array = array[i:j]
#             if tuple(sub_array) not in have_list:
#                 have_list.add(tuple(sub_array))


#             k = 1
#             while k < len(sub_array) - 1:
#                 del sub_array[k]
#                 if tuple(sub_array) not in have_list:
#                     pass
#                 subsequence(sub_array)
#                 k += 1
#             j -= 1

# arr = ('ASPNHSUS',
#  'UNRATE',
#  'USACPIHOUMINMEI',
#  'FEDFUNDS',
#  'CSUSHPINSA',
#  'MSPNHSUS', 'MSACSR',
#  'MORTGAGE30US',
#  'NHFSEPUC',
#  'NHSDPC',
#        'NHSDPNS',
#        'HNFSEPUSSA',
#        'HSN1F',
#        'DSPIC96',
#        'TOTALSA',
#        'DIFSRC1Q027SBEA',
#        'W774RC1Q027SBEA',
#        'Sales in $MM'
# )
# # Run the above algorithm to find all possible subsequence of the features
# subsequence(arr)

# #### !!!!! BECAUSE THE ABOVE ALGORITHM TAKES A VERY LONG TIME TO RUN (AROUND 15 MINUTES OR MORE) WE SAVE THE OUTPUT
# #### BY HARD CODING THE OUTPUTS OF ALL POSSIBLE FEATURE SUBSEQUENCE INTO THE HAVE_LIST SET. 
have_list = {('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'HSN1F', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSACSR', 'MORTGAGE30US'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS'), ('MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE',), ('MORTGAGE30US', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR',), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA'), ('MSACSR', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA'), ('NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'HSN1F'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPNS', 'Sales in $MM'), ('CSUSHPINSA', 'HSN1F', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HNFSEPUSSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHSDPC'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPC'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'CSUSHPINSA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'NHSDPNS'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHFSEPUC'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPNS', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('FEDFUNDS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F'), ('MORTGAGE30US', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'HSN1F', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US'), ('NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'DSPIC96', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('DSPIC96',), ('FEDFUNDS', 'DSPIC96', 'W774RC1Q027SBEA'), ('NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US'), ('CSUSHPINSA', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HSN1F'), ('MSPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'HSN1F', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'DSPIC96', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA'), ('DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F'), ('UNRATE', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96'), ('NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('NHFSEPUC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('MSPNHSUS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHFSEPUC', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('NHSDPNS', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'DSPIC96', 'Sales in $MM'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'TOTALSA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'NHSDPC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHFSEPUC'), ('ASPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('NHFSEPUC', 'DSPIC96', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('MSACSR', 'TOTALSA'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HSN1F', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'MORTGAGE30US'), ('FEDFUNDS', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US'), ('MORTGAGE30US', 'TOTALSA'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'Sales in $MM'), ('HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('UNRATE', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'W774RC1Q027SBEA'), ('HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'TOTALSA', 'Sales in $MM'), ('NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'TOTALSA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC'), ('NHFSEPUC', 'HSN1F'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPNS'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MORTGAGE30US', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPC'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'DSPIC96', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MSACSR'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'TOTALSA'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('MSPNHSUS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'NHFSEPUC', 'HSN1F'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC'), ('UNRATE', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPNS', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC'), ('CSUSHPINSA', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR'), ('MSPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS'), ('MORTGAGE30US', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHFSEPUC', 'HSN1F', 'TOTALSA'), ('MSPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS'), ('MSPNHSUS', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'NHSDPNS'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'DSPIC96'), ('MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA'), ('DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA'), ('CSUSHPINSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96'), ('UNRATE', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPNS'), ('NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HNFSEPUSSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('MSACSR', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'HNFSEPUSSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC'), ('UNRATE', 'HNFSEPUSSA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('DIFSRC1Q027SBEA',), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'MORTGAGE30US'), ('HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'TOTALSA'), ('NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR'), ('MSACSR', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'USACPIHOUMINMEI'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPNS', 'DSPIC96'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'DSPIC96', 'TOTALSA'), ('NHFSEPUC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPNS', 'TOTALSA'), ('MSACSR', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'DSPIC96', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPC'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('NHFSEPUC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPNS', 'DSPIC96'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR'), ('NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA'), ('FEDFUNDS', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('CSUSHPINSA', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('USACPIHOUMINMEI', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'W774RC1Q027SBEA'), ('NHSDPC', 'HSN1F'), ('NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('HNFSEPUSSA',), ('HSN1F', 'TOTALSA'), ('MSPNHSUS', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'FEDFUNDS'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('FEDFUNDS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'TOTALSA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('UNRATE', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPC', 'NHSDPNS'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'DSPIC96'), ('UNRATE', 'TOTALSA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('CSUSHPINSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'TOTALSA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'TOTALSA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'NHSDPC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'HNFSEPUSSA', 'DSPIC96'), ('MSACSR', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS',), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('TOTALSA',), ('NHSDPNS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('FEDFUNDS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MSPNHSUS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F'), ('NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('MSPNHSUS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'HSN1F', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('MSACSR', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'TOTALSA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'FEDFUNDS'), ('FEDFUNDS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHFSEPUC'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'TOTALSA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS', 'DSPIC96'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'HSN1F'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'HNFSEPUSSA', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'HSN1F', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('HSN1F',), ('HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'DSPIC96', 'TOTALSA'), ('DSPIC96', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('HNFSEPUSSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('CSUSHPINSA', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA'), ('MSACSR', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA'), ('NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC'), ('MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('NHFSEPUC', 'HNFSEPUSSA', 'HSN1F'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('FEDFUNDS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'TOTALSA'), ('HSN1F', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHFSEPUC', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'DSPIC96', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'HSN1F'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA',), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'HSN1F', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS'), ('MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DIFSRC1Q027SBEA'), ('NHFSEPUC', 'TOTALSA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('ASPNHSUS', 'NHSDPC', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'HNFSEPUSSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHFSEPUC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('NHSDPC', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'HSN1F'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPC'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F'), ('FEDFUNDS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'DSPIC96'), ('TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'HNFSEPUSSA', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHFSEPUC', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'HSN1F', 'DSPIC96'), ('MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('FEDFUNDS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'TOTALSA'), ('CSUSHPINSA', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPNS', 'HSN1F'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HSN1F'), ('NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'DIFSRC1Q027SBEA'), ('MSACSR', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HNFSEPUSSA', 'TOTALSA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'NHSDPNS', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'TOTALSA'), ('MORTGAGE30US', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'DSPIC96'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'HSN1F', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('HNFSEPUSSA', 'TOTALSA'), ('NHFSEPUC', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HSN1F'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA'), ('UNRATE', 'NHSDPC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HSN1F', 'TOTALSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'TOTALSA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('MSPNHSUS', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('MORTGAGE30US', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHFSEPUC', 'DSPIC96', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'DSPIC96', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'Sales in $MM'), ('MORTGAGE30US', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US'), ('DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('MSACSR', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'HNFSEPUSSA', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'HNFSEPUSSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'HSN1F', 'DSPIC96'), ('MORTGAGE30US', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('UNRATE', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'TOTALSA'), ('ASPNHSUS', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPNS', 'HNFSEPUSSA'), ('NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHFSEPUC'), ('MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('ASPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPC'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US'), ('FEDFUNDS', 'MSPNHSUS'), ('CSUSHPINSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'HSN1F'), ('MSACSR', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('CSUSHPINSA', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HSN1F', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('MSACSR', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS',), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA'), ('NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('UNRATE', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US'), ('FEDFUNDS', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'HSN1F', 'DSPIC96'), ('NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHFSEPUC'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MSACSR', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F'), ('UNRATE', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('MSACSR', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('NHSDPNS', 'HSN1F', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHSDPNS', 'TOTALSA'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HSN1F', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('MSACSR', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('NHSDPNS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'DSPIC96', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC'), ('NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F'), ('NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC',), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('NHSDPNS',), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPC'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'DSPIC96'), ('ASPNHSUS', 'TOTALSA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'NHFSEPUC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA'), ('UNRATE', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US'), ('ASPNHSUS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HNFSEPUSSA'), ('FEDFUNDS', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'DSPIC96', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC'), ('NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC'), ('NHSDPC', 'DSPIC96', 'TOTALSA'), ('NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'UNRATE'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS'), ('USACPIHOUMINMEI', 'DSPIC96', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HSN1F', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHSDPNS', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'NHSDPC', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPC'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'NHSDPNS', 'DSPIC96'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US'), ('MSACSR', 'HSN1F', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA'), ('HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPC', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'HSN1F', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPC'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F'), ('MORTGAGE30US', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPNS', 'HSN1F', 'DSPIC96'), ('TOTALSA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSACSR', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA'), ('MSACSR', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'TOTALSA'), ('MSPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DSPIC96'), ('CSUSHPINSA', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPC'), ('MSPNHSUS', 'HSN1F'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHSDPNS', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC'), ('UNRATE', 'NHSDPNS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('FEDFUNDS',), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'DSPIC96'), ('FEDFUNDS', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('DSPIC96', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('MSACSR', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'NHSDPNS', 'TOTALSA'), ('FEDFUNDS', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS'), ('NHSDPNS', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('MSPNHSUS', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'TOTALSA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHSDPC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC'), ('NHFSEPUC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPNS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'HSN1F', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('MSPNHSUS', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'DSPIC96'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'DSPIC96', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F'), ('MORTGAGE30US', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'DSPIC96', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'DSPIC96'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MSACSR', 'DSPIC96'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('DIFSRC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'HNFSEPUSSA'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'TOTALSA', 'Sales in $MM'), ('CSUSHPINSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSACSR'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('HSN1F', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'DSPIC96'), ('UNRATE', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'NHFSEPUC'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('MSACSR', 'DSPIC96', 'Sales in $MM'), ('MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'Sales in $MM'), ('CSUSHPINSA', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'DSPIC96', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('MSPNHSUS', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('CSUSHPINSA', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHSDPC', 'HSN1F', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('HNFSEPUSSA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHSDPC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'HNFSEPUSSA'), ('MSPNHSUS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('MSPNHSUS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'DSPIC96'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96'), ('ASPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DSPIC96'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('NHFSEPUC', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC'), ('NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'DSPIC96'), ('MORTGAGE30US', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('CSUSHPINSA', 'TOTALSA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'NHSDPC', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHSDPC'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MSPNHSUS'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHFSEPUC', 'NHSDPC'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F'), ('MORTGAGE30US', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI',), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HSN1F', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'HNFSEPUSSA', 'DSPIC96'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'MSACSR'), ('MSPNHSUS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('Sales in $MM',), ('ASPNHSUS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('USACPIHOUMINMEI', 'MSPNHSUS'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('FEDFUNDS', 'HNFSEPUSSA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'TOTALSA'), ('FEDFUNDS', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPNS', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPNS', 'TOTALSA'), ('MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'DSPIC96', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('HSN1F', 'DSPIC96'), ('MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHSDPC', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('UNRATE', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('FEDFUNDS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPNS', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSACSR', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'NHSDPC', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA'), ('UNRATE', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHSDPNS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'HSN1F'), ('UNRATE', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHSDPC', 'HSN1F'), ('NHFSEPUC',), ('CSUSHPINSA', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'HSN1F'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA'), ('CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('ASPNHSUS', 'NHSDPNS', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA'), ('MSPNHSUS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'TOTALSA', 'Sales in $MM'), ('HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'Sales in $MM'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'DSPIC96'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('DSPIC96', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('NHSDPC', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'NHFSEPUC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'TOTALSA', 'Sales in $MM'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'TOTALSA'), ('ASPNHSUS', 'NHSDPC', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('MSPNHSUS', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA'), ('UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'UNRATE', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC'), ('NHFSEPUC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HSN1F', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('MSACSR', 'Sales in $MM'), ('MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('NHSDPC', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS'), ('USACPIHOUMINMEI', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHSDPNS', 'DSPIC96'), ('UNRATE', 'NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPNS', 'HNFSEPUSSA'), ('MORTGAGE30US', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('FEDFUNDS', 'DSPIC96', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'HNFSEPUSSA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HSN1F'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US'), ('ASPNHSUS', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('HNFSEPUSSA', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('UNRATE', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'DSPIC96'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPNS', 'TOTALSA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('MSPNHSUS', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('CSUSHPINSA', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHSDPC', 'W774RC1Q027SBEA'), ('NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HSN1F', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MORTGAGE30US', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHFSEPUC', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('FEDFUNDS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('NHSDPNS', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US',), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHSDPNS', 'HSN1F'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'NHFSEPUC', 'TOTALSA', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HNFSEPUSSA', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHFSEPUC'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA'), ('MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS'), ('UNRATE', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA'), ('NHSDPC', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('NHFSEPUC', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MORTGAGE30US'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('MORTGAGE30US', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('MSACSR', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSPNHSUS', 'NHSDPC', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS', 'DIFSRC1Q027SBEA'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS'), ('HNFSEPUSSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('W774RC1Q027SBEA',), ('CSUSHPINSA', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'HSN1F'), ('UNRATE', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHSDPNS', 'TOTALSA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'HSN1F', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F'), ('USACPIHOUMINMEI', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('FEDFUNDS', 'NHSDPC', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'TOTALSA'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US'), ('UNRATE', 'MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC'), ('NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MORTGAGE30US', 'DSPIC96'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MSPNHSUS', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('FEDFUNDS', 'HSN1F'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('NHSDPC', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('HSN1F', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('USACPIHOUMINMEI', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'DSPIC96'), ('HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('NHSDPC', 'TOTALSA'), ('NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'FEDFUNDS', 'CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC'), ('UNRATE', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('MSPNHSUS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'NHSDPC', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC'), ('MSACSR', 'NHSDPC'), ('FEDFUNDS', 'MORTGAGE30US', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'DSPIC96', 'DIFSRC1Q027SBEA'), ('FEDFUNDS', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA'), ('MSACSR', 'NHSDPNS', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHFSEPUC', 'NHSDPNS', 'HSN1F'), ('USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('NHFSEPUC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MORTGAGE30US', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS'), ('ASPNHSUS', 'MSPNHSUS', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSACSR', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('MSPNHSUS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA'), ('NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('ASPNHSUS', 'HNFSEPUSSA', 'Sales in $MM'), ('NHFSEPUC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPNS'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('MSACSR', 'TOTALSA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHFSEPUC', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'HNFSEPUSSA', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('ASPNHSUS', 'NHSDPC', 'HNFSEPUSSA'), ('USACPIHOUMINMEI', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'USACPIHOUMINMEI', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('FEDFUNDS', 'NHSDPNS', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('MSPNHSUS', 'NHSDPC', 'HSN1F'), ('MORTGAGE30US', 'HSN1F', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('CSUSHPINSA', 'MSACSR'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('FEDFUNDS', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('UNRATE', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'NHFSEPUC', 'HSN1F', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('CSUSHPINSA', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA'), ('USACPIHOUMINMEI', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('UNRATE', 'HSN1F', 'Sales in $MM'), ('MSPNHSUS', 'NHSDPNS', 'HSN1F', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('UNRATE', 'MSACSR', 'NHSDPNS', 'HSN1F', 'TOTALSA'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA'), ('ASPNHSUS', 'FEDFUNDS', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('UNRATE', 'MORTGAGE30US', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('UNRATE', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA'), ('NHSDPC', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('USACPIHOUMINMEI', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('UNRATE', 'MSPNHSUS', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'NHSDPC', 'DSPIC96'), ('NHFSEPUC', 'TOTALSA'), ('ASPNHSUS', 'MSACSR', 'NHFSEPUC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSPNHSUS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('ASPNHSUS', 'CSUSHPINSA', 'NHFSEPUC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'NHSDPNS', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('MSACSR', 'NHFSEPUC'), ('HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('USACPIHOUMINMEI', 'MORTGAGE30US', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('CSUSHPINSA', 'MSPNHSUS', 'MSACSR', 'MORTGAGE30US', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F'), ('ASPNHSUS', 'MSACSR', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96'), ('FEDFUNDS', 'HSN1F', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96', 'TOTALSA', 'DIFSRC1Q027SBEA'), ('HSN1F', 'W774RC1Q027SBEA', 'Sales in $MM'), ('ASPNHSUS', 'CSUSHPINSA', 'MORTGAGE30US', 'NHSDPC', 'HNFSEPUSSA', 'DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM'), ('USACPIHOUMINMEI', 'MSACSR', 'NHFSEPUC', 'NHSDPC', 'NHSDPNS', 'HNFSEPUSSA', 'HSN1F', 'DSPIC96'), ('FEDFUNDS', 'MSPNHSUS', 'MORTGAGE30US', 'NHSDPC', 'NHSDPNS')}


### 1. Run the Support Vector Machine (SVM) model
* Best Accuracy: 85.629999999999995%
* Features: ['FEDFUNDS', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA']

In [95]:
percentage = [] # used to SAVE and contain the combination of features that yields the best performance
# However, as we save the whole column, it WILL definitely include our target variable (feature) 'Sales in $MM',
# but in actual when we train the model, it will not be included.

p_map = {} # Save all the combinations that have a relative high performance and their test and trained scale for double checking our program and to calculate the RMSE 
largest_percentage = float("-inf") # a flaging variable used to record the global maximun (BEST) performance accuracy
# Loop through all our possible combinations of features
for columns in have_list:
    columns = list(columns)
    # create a set of the features for faster search, look up
    col_set = set(columns)
    # as dataframes with 0 or 1 columns cause a bug for scaling (need at least 2) to ignore such possible combination
    if len(columns) == 0 or len(columns) == 1:
        continue
    # AS the 'Sales in $MM' column is the target variable, we MUST have it in our combination, so if its not, add it in
    if 'Sales in $MM' not in col_set:
        columns.append('Sales in $MM')
    
    # Recreate, use the dataframe copy incase it changes our original dataframes, and create training and testing
    # base on the combination of features in the current loop
    df_train_features = df_train.copy()
    df_train_features = df_train_features[columns]
    df_test_features = df_test.copy()
    df_test_features = df_test_features[columns]
    # delete the "DATE" column, use try-except clause in case there is no such column in the current combination
    try:
        del df_test_features["DATE"]
        del df_train_features["DATE"]
    except:
        pass
    
    # This is used to record the index value that we would have to drop later for scaling and modeling as columns
    # after scale would become their corresponding column index in the data type of an integer
    target = columns.index('Sales in $MM')
    
    # And then run the rest of the model
    min_max_scaler = preprocessing.MinMaxScaler()
    #generalization technique to put all pandas to more acceptable range so that all columns are in the similar range

    train_scaled_parse = min_max_scaler.fit_transform(df_train_features)
    #fit the model based on scale
    train_scaled = pd.DataFrame(train_scaled_parse)#create dataframe

    X,y=train_scaled.drop([target], axis=1), train_scaled[target]#separate features and target variable
    #X_test,y_test = df_test.drop(['Sales in $MM'], axis=1), df_test['Sales in $MM']
    min_max_scaler = preprocessing.MinMaxScaler()#repeating the same procedure for testing set

    test_scaled_parse = min_max_scaler.fit_transform(df_test_features)
    test_scaled = pd.DataFrame(test_scaled_parse)


    X_test,y_test = test_scaled.drop([target], axis=1), test_scaled[target]

    clf= svm.SVR()#to train the model using SVM because of the linear trend
    #lab_enc = preprocessing.LabelEncoder()
    clf.fit(X, y)#fitting the algorithm using the training model

    acc_SvmSvr =round(clf.score(X_test, y_test)*100,2)
    #print(str(acc_SvmSvr)+'percent')
    if acc_SvmSvr > largest_percentage:
        largest_percentage = acc_SvmSvr
        percentage = columns
        p_map[acc_SvmSvr] = columns, X_test, y_test
    #percentage.append(acc_SvmSvr)
    #using the test variables, to predict the target and compare it with the test y

In [96]:
p_map

{79.099999999999994: (['CSUSHPINSA',
   'MORTGAGE30US',
   'NHSDPC',
   'HNFSEPUSSA',
   'HSN1F',
   'DSPIC96',
   'TOTALSA',
   'DIFSRC1Q027SBEA',
   'W774RC1Q027SBEA',
   'Sales in $MM'],
             0         1         2         3         4         5     6  \
  0   0.000000  0.573394  0.000000  0.000000  0.000000  0.028217  0.68   
  1   0.011624  0.294889  0.333333  0.051724  0.026178  0.000000  0.64   
  2   0.074299  0.339450  0.333333  0.103448  0.068063  0.095664  0.32   
  3   0.164018  0.222805  0.555556  0.068966  0.240838  0.193393  0.60   
  4   0.252218  0.216252  0.555556  0.068966  0.209424  0.244666  0.48   
  5   0.327940  0.174312  0.555556  0.103448  0.204188  0.300413  0.40   
  6   0.381364  0.006553  0.555556  0.000000  0.560209  0.370957  0.68   
  7   0.412393  0.000000  0.222222  0.068966  0.246073  0.310736  0.44   
  8   0.426674  0.032765  0.222222  0.086207  0.261780  0.284240  0.64   
  9   0.430517  0.045872  0.555556  0.189655  0.298429  0.198555  0.72

In [82]:
largest_percentage

85.629999999999995

In [83]:
percentage

[['FEDFUNDS', 'HNFSEPUSSA', 'TOTALSA', 'W774RC1Q027SBEA', 'Sales in $MM']]

In [161]:
# AS the rest are all the same code except using different a model, the logic flow is all the same

### 2. Run the BayesianRidge model
* Best Accuracy: 84.700000000000003%
* Features: ['DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA']

In [101]:
percentage = []
p_map = {}
largest_percentage = float("-inf")
for columns in have_list:
    columns = list(columns)
    col_set = set(columns)
    if len(columns) == 0 or len(columns) == 1:
        continue
    if 'Sales in $MM' not in col_set:
        columns.append('Sales in $MM')
#     else:
#         del columns[columns.index('Sales in $MM')]
#         columns.append('Sales in $MM') # move it to the end
    
    df_train_features = df_train.copy()
    df_train_features = df_train_features[columns]
    df_test_features = df_test.copy()
    df_test_features = df_test_features[columns]
    try:
        del df_test_features["DATE"]
        del df_train_features["DATE"]
    except:
        pass
    
    target = columns.index('Sales in $MM')
    
    # And then run the rest of the model

    min_max_scaler = preprocessing.MinMaxScaler()
    #generalization technique to put all pandas to more acceptable range so that all columns are in the similar range

    train_scaled_parse = min_max_scaler.fit_transform(df_train_features)
    #fit the model based on scale
    train_scaled = pd.DataFrame(train_scaled_parse)#create dataframe

    X,y=train_scaled.drop([target], axis=1), train_scaled[target]#separate features and target variable
    #X_test,y_test = df_test.drop(['Sales in $MM'], axis=1), df_test['Sales in $MM']
    min_max_scaler = preprocessing.MinMaxScaler()#repeating the same procedure for testing set

    test_scaled_parse = min_max_scaler.fit_transform(df_test_features)
    test_scaled = pd.DataFrame(test_scaled_parse)


    X_test,y_test = test_scaled.drop([target], axis=1), test_scaled[target]

    clf= BayesianRidge(n_iter= 500)#to train the model using BayesianRidge because of the linear trend
    #lab_enc = preprocessing.LabelEncoder()
    clf.fit(X, y)#fitting the algorithm using the training model

    acc_SvmSvr =round(clf.score(X_test, y_test)*100,2)
    #print(str(acc_SvmSvr)+'percent')
    if acc_SvmSvr > largest_percentage:
        largest_percentage = acc_SvmSvr
        percentage = columns
        p_map[acc_SvmSvr] = columns, X_test, y_test
    #percentage.append(acc_SvmSvr)
    #using the test variables, to predict the target and compare it with the test y

In [102]:
percentage

['DSPIC96', 'DIFSRC1Q027SBEA', 'W774RC1Q027SBEA', 'Sales in $MM']

In [104]:
p_map

{68.280000000000001: (['CSUSHPINSA',
   'MORTGAGE30US',
   'NHSDPC',
   'HNFSEPUSSA',
   'HSN1F',
   'DSPIC96',
   'TOTALSA',
   'DIFSRC1Q027SBEA',
   'W774RC1Q027SBEA',
   'Sales in $MM'],
             0         1         2         3         4         5     6  \
  0   0.000000  0.573394  0.000000  0.000000  0.000000  0.028217  0.68   
  1   0.011624  0.294889  0.333333  0.051724  0.026178  0.000000  0.64   
  2   0.074299  0.339450  0.333333  0.103448  0.068063  0.095664  0.32   
  3   0.164018  0.222805  0.555556  0.068966  0.240838  0.193393  0.60   
  4   0.252218  0.216252  0.555556  0.068966  0.209424  0.244666  0.48   
  5   0.327940  0.174312  0.555556  0.103448  0.204188  0.300413  0.40   
  6   0.381364  0.006553  0.555556  0.000000  0.560209  0.370957  0.68   
  7   0.412393  0.000000  0.222222  0.068966  0.246073  0.310736  0.44   
  8   0.426674  0.032765  0.222222  0.086207  0.261780  0.284240  0.64   
  9   0.430517  0.045872  0.555556  0.189655  0.298429  0.198555  0.72

In [103]:
largest_percentage

84.700000000000003

### 3. Run the RandomForestRegressor model
* Best Accuracy: 86.390000000000001%
* Features: ['MORTGAGE30US',
 'HNFSEPUSSA',
 'TOTALSA',
 'DIFSRC1Q027SBEA',
 'W774RC1Q027SBEA']

In [152]:
percentage = []
p_map = {}
largest_percentage = float("-inf")
for columns in have_list:
    columns = list(columns)
    col_set = set(columns)
    if len(columns) == 0 or len(columns) == 1:
        continue
    if 'Sales in $MM' not in col_set:
        columns.append('Sales in $MM')
#     else:
#         del columns[columns.index('Sales in $MM')]
#         columns.append('Sales in $MM') # move it to the end
    
    df_train_features = df_train.copy()
    df_train_features = df_train_features[columns]
    df_test_features = df_test.copy()
    df_test_features = df_test_features[columns]
    try:
        del df_test_features["DATE"]
        del df_train_features["DATE"]
    except:
        pass
    
    target = columns.index('Sales in $MM')
    
    # And then run the rest of the model

    min_max_scaler = preprocessing.MinMaxScaler()
    #generalization technique to put all pandas to more acceptable range so that all columns are in the similar range

    train_scaled_parse = min_max_scaler.fit_transform(df_train_features)
    #fit the model based on scale
    train_scaled = pd.DataFrame(train_scaled_parse)#create dataframe

    X,y=train_scaled.drop([target], axis=1), train_scaled[target]#separate features and target variable
    #X_test,y_test = df_test.drop(['Sales in $MM'], axis=1), df_test['Sales in $MM']
    min_max_scaler = preprocessing.MinMaxScaler()#repeating the same procedure for testing set

    test_scaled_parse = min_max_scaler.fit_transform(df_test_features)
    test_scaled = pd.DataFrame(test_scaled_parse)


    X_test,y_test = test_scaled.drop([target], axis=1), test_scaled[target]

    clf= RandomForestRegressor()#n_estimators=500
    #lab_enc = preprocessing.LabelEncoder()
    clf.fit(X, y)#fitting the algorithm using the training model

    acc_SvmSvr =round(clf.score(X_test, y_test)*100,2)
    
    #print(str(acc_SvmSvr)+'percent')
    if acc_SvmSvr > largest_percentage:
        largest_percentage = acc_SvmSvr
        percentage = columns
        p_map[acc_SvmSvr] = columns, X_test, y_test
    #percentage.append(acc_SvmSvr)
    #using the test variables, to predict the target and compare it with the test y

In [153]:
percentage

['MORTGAGE30US',
 'HNFSEPUSSA',
 'TOTALSA',
 'DIFSRC1Q027SBEA',
 'W774RC1Q027SBEA',
 'Sales in $MM']

In [154]:
largest_percentage

86.390000000000001

In [155]:
p_map

{69.799999999999997: (['CSUSHPINSA',
   'MORTGAGE30US',
   'NHSDPC',
   'HNFSEPUSSA',
   'HSN1F',
   'DSPIC96',
   'TOTALSA',
   'DIFSRC1Q027SBEA',
   'W774RC1Q027SBEA',
   'Sales in $MM'],
             0         1         2         3         4         5     6  \
  0   0.000000  0.573394  0.000000  0.000000  0.000000  0.028217  0.68   
  1   0.011624  0.294889  0.333333  0.051724  0.026178  0.000000  0.64   
  2   0.074299  0.339450  0.333333  0.103448  0.068063  0.095664  0.32   
  3   0.164018  0.222805  0.555556  0.068966  0.240838  0.193393  0.60   
  4   0.252218  0.216252  0.555556  0.068966  0.209424  0.244666  0.48   
  5   0.327940  0.174312  0.555556  0.103448  0.204188  0.300413  0.40   
  6   0.381364  0.006553  0.555556  0.000000  0.560209  0.370957  0.68   
  7   0.412393  0.000000  0.222222  0.068966  0.246073  0.310736  0.44   
  8   0.426674  0.032765  0.222222  0.086207  0.261780  0.284240  0.64   
  9   0.430517  0.045872  0.555556  0.189655  0.298429  0.198555  0.72

In [156]:
p_map[largest_percentage][1]

,0,1,2,3,4
0,0.573394,0.000000,0.68,0.000000,0.000000
1,0.294889,0.051724,0.64,0.000000,0.000000
2,0.339450,0.103448,0.32,0.000000,0.000000
3,0.222805,0.068966,0.60,0.103853,0.103853
4,0.216252,0.068966,0.48,0.103853,0.103853
5,0.174312,0.103448,0.40,0.103853,0.103853
6,0.006553,0.000000,0.68,0.272195,0.272195
7,0.000000,0.068966,0.44,0.272195,0.272195
8,0.032765,0.086207,0.64,0.272195,0.272195
9,0.045872,0.189655,0.72,0.367048,0.367048


# We Calculate the RMSE for the RandomForestRegressor model as it yields the best accuracy (up to 87.23%, however it sometimes changes when we rerun it several times in the same condition)
* RMSE Value = 0.12436532435830513

In [157]:
y_pred_Svmsvr = clf.predict(p_map[largest_percentage][1])

In [158]:
# calculate RMSE for the best (Highest) accuracy prediction with an accuracy of: 87.23% --> RandomForest
# Best RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
rms_svm = sqrt(mean_squared_error(y_test,y_pred_Svmsvr ))
rms_svm

0.12436532435830513

### 4. Run the DecisionTreeRegressor model
* Best Accuracy: 85.219999999999999%
* Features: ['MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA']

In [110]:
percentage = []
p_map = {}
largest_percentage = float("-inf")
for columns in have_list:
    columns = list(columns)
    col_set = set(columns)
    if len(columns) == 0 or len(columns) == 1:
        continue
    if 'Sales in $MM' not in col_set:
        columns.append('Sales in $MM')
#     else:
#         del columns[columns.index('Sales in $MM')]
#         columns.append('Sales in $MM') # move it to the end
    
    df_train_features = df_train.copy()
    df_train_features = df_train_features[columns]
    df_test_features = df_test.copy()
    df_test_features = df_test_features[columns]
    try:
        del df_test_features["DATE"]
        del df_train_features["DATE"]
    except:
        pass
    
    target = columns.index('Sales in $MM')
    
    # And then run the rest of the model

    min_max_scaler = preprocessing.MinMaxScaler()
    #generalization technique to put all pandas to more acceptable range so that all columns are in the similar range

    train_scaled_parse = min_max_scaler.fit_transform(df_train_features)
    #fit the model based on scale
    train_scaled = pd.DataFrame(train_scaled_parse)#create dataframe

    X,y=train_scaled.drop([target], axis=1), train_scaled[target]#separate features and target variable
    #X_test,y_test = df_test.drop(['Sales in $MM'], axis=1), df_test['Sales in $MM']
    min_max_scaler = preprocessing.MinMaxScaler()#repeating the same procedure for testing set

    test_scaled_parse = min_max_scaler.fit_transform(df_test_features)
    test_scaled = pd.DataFrame(test_scaled_parse)


    X_test,y_test = test_scaled.drop([target], axis=1), test_scaled[target]

    clf= tree.DecisionTreeRegressor()#to train the model using DecisionTreeRegressor
    #lab_enc = preprocessing.LabelEncoder()
    clf.fit(X, y)#fitting the algorithm using the training model

    acc_SvmSvr =round(clf.score(X_test, y_test)*100,2)
    #print(str(acc_SvmSvr)+'percent')
    if acc_SvmSvr > largest_percentage:
        largest_percentage = acc_SvmSvr
        percentage = columns
        p_map[acc_SvmSvr] = columns, X_test, y_test
    #percentage.append(acc_SvmSvr)
    #using the test variables, to predict the target and compare it with the test y

In [111]:
percentage

['MORTGAGE30US', 'HNFSEPUSSA', 'TOTALSA', 'DIFSRC1Q027SBEA', 'Sales in $MM']

In [113]:
largest_percentage

85.219999999999999

In [112]:
p_map

{63.68: (['CSUSHPINSA',
   'MORTGAGE30US',
   'NHSDPC',
   'HNFSEPUSSA',
   'HSN1F',
   'DSPIC96',
   'TOTALSA',
   'DIFSRC1Q027SBEA',
   'W774RC1Q027SBEA',
   'Sales in $MM'],
             0         1         2         3         4         5     6  \
  0   0.000000  0.573394  0.000000  0.000000  0.000000  0.028217  0.68   
  1   0.011624  0.294889  0.333333  0.051724  0.026178  0.000000  0.64   
  2   0.074299  0.339450  0.333333  0.103448  0.068063  0.095664  0.32   
  3   0.164018  0.222805  0.555556  0.068966  0.240838  0.193393  0.60   
  4   0.252218  0.216252  0.555556  0.068966  0.209424  0.244666  0.48   
  5   0.327940  0.174312  0.555556  0.103448  0.204188  0.300413  0.40   
  6   0.381364  0.006553  0.555556  0.000000  0.560209  0.370957  0.68   
  7   0.412393  0.000000  0.222222  0.068966  0.246073  0.310736  0.44   
  8   0.426674  0.032765  0.222222  0.086207  0.261780  0.284240  0.64   
  9   0.430517  0.045872  0.555556  0.189655  0.298429  0.198555  0.72   
  10  0.4

### 5. Run the GradientBoostingRegressor model
* Best Accuracy: 86.049999999999997%
* Features: ['USACPIHOUMINMEI',
 'MORTGAGE30US',
 'HNFSEPUSSA',
 'DSPIC96',
 'DIFSRC1Q027SBEA',
 'W774RC1Q027SBEA']

In [115]:
percentage = []
p_map = {}
largest_percentage = float("-inf")
for columns in have_list:
    columns = list(columns)
    col_set = set(columns)
    if len(columns) == 0 or len(columns) == 1:
        continue
    if 'Sales in $MM' not in col_set:
        columns.append('Sales in $MM')
#     else:
#         del columns[columns.index('Sales in $MM')]
#         columns.append('Sales in $MM') # move it to the end
    
    df_train_features = df_train.copy()
    df_train_features = df_train_features[columns]
    df_test_features = df_test.copy()
    df_test_features = df_test_features[columns]
    try:
        del df_test_features["DATE"]
        del df_train_features["DATE"]
    except:
        pass
    
    target = columns.index('Sales in $MM')
    
    # And then run the rest of the model

    min_max_scaler = preprocessing.MinMaxScaler()
    #generalization technique to put all pandas to more acceptable range so that all columns are in the similar range

    train_scaled_parse = min_max_scaler.fit_transform(df_train_features)
    #fit the model based on scale
    train_scaled = pd.DataFrame(train_scaled_parse)#create dataframe

    X,y=train_scaled.drop([target], axis=1), train_scaled[target]#separate features and target variable
    #X_test,y_test = df_test.drop(['Sales in $MM'], axis=1), df_test['Sales in $MM']
    min_max_scaler = preprocessing.MinMaxScaler()#repeating the same procedure for testing set

    test_scaled_parse = min_max_scaler.fit_transform(df_test_features)
    test_scaled = pd.DataFrame(test_scaled_parse)


    X_test,y_test = test_scaled.drop([target], axis=1), test_scaled[target]

    clf= GradientBoostingRegressor( )#n_estimators=500
    #lab_enc = preprocessing.LabelEncoder()
    clf.fit(X, y)#fitting the algorithm using the training model

    acc_SvmSvr =round(clf.score(X_test, y_test)*100,2)
    #print(str(acc_SvmSvr)+'percent')
    if acc_SvmSvr > largest_percentage:
        largest_percentage = acc_SvmSvr
        percentage = columns
        p_map[acc_SvmSvr] = columns, X_test, y_test
    #percentage.append(acc_SvmSvr)
    #using the test variables, to predict the target and compare it with the test y

In [116]:
percentage

['USACPIHOUMINMEI',
 'MORTGAGE30US',
 'HNFSEPUSSA',
 'DSPIC96',
 'DIFSRC1Q027SBEA',
 'W774RC1Q027SBEA',
 'Sales in $MM']

In [118]:
largest_percentage

86.049999999999997

In [117]:
p_map

{72.060000000000002: (['CSUSHPINSA',
   'MORTGAGE30US',
   'NHSDPC',
   'HNFSEPUSSA',
   'HSN1F',
   'DSPIC96',
   'TOTALSA',
   'DIFSRC1Q027SBEA',
   'W774RC1Q027SBEA',
   'Sales in $MM'],
             0         1         2         3         4         5     6  \
  0   0.000000  0.573394  0.000000  0.000000  0.000000  0.028217  0.68   
  1   0.011624  0.294889  0.333333  0.051724  0.026178  0.000000  0.64   
  2   0.074299  0.339450  0.333333  0.103448  0.068063  0.095664  0.32   
  3   0.164018  0.222805  0.555556  0.068966  0.240838  0.193393  0.60   
  4   0.252218  0.216252  0.555556  0.068966  0.209424  0.244666  0.48   
  5   0.327940  0.174312  0.555556  0.103448  0.204188  0.300413  0.40   
  6   0.381364  0.006553  0.555556  0.000000  0.560209  0.370957  0.68   
  7   0.412393  0.000000  0.222222  0.068966  0.246073  0.310736  0.44   
  8   0.426674  0.032765  0.222222  0.086207  0.261780  0.284240  0.64   
  9   0.430517  0.045872  0.555556  0.189655  0.298429  0.198555  0.72